In [2]:
import pandas as pd
import numpy as np
import boto3
import sagemaker
import os
import re
import io
import nltk
from nltk.corpus import stopwords
import math
import gzip
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
region = boto3.Session().region_name
role = sagemaker.get_execution_role()

bucket='dlsg-training-data-prod'
prefix = 'a708148/' 
kms_key = "arn:aws:kms:us-east-1:647324198242:alias/ap131630-sagemaker"
sagemaker_session = sagemaker.Session()

s3_input      = 's3://'+bucket+'/'+prefix+'Rec_sys_challenge/original_dataset/'
s3_output_location  = 's3://'+bucket+'/'+prefix+'Rec_sys_challenge/output/'

In [4]:
articles = sagemaker.s3.S3Downloader.read_file('s3://dlsg-training-data-prod/a708148/Rec_sys_challenge/original_dataset/articles.csv', sagemaker_session)
articles = pd.read_csv(io.StringIO(articles), sep=",")

In [5]:
users = sagemaker.s3.S3Downloader.read_file('s3://dlsg-training-data-prod/a708148/Rec_sys_challenge/original_dataset/users_v3.csv', sagemaker_session)
users = pd.read_csv(io.StringIO(users), sep=",")

In [6]:
df = sagemaker.s3.S3Downloader.read_file('s3://dlsg-training-data-prod/a708148/Rec_sys_challenge/original_dataset/Rec_Sys_sample_train_data_v3.csv', sagemaker_session)
df = pd.read_csv(io.StringIO(df), sep=",")

In [7]:
article_id=articles['tcm_id'].unique()

In [8]:
np.savetxt('./data/Challenge_Dataset/articles.txt',article_id,fmt="%s")
np.savetxt('./data/Challenge_Dataset/related_article.txt',article_id,fmt="%s")

In [9]:
topic_list = []
topics = articles['topic'].dropna()
for topic in topics:
    t = topic.strip('[').strip(']').strip().split(',')
    for i in range(len(t)):
        j = t[i].strip().replace("'",'')
        if j not in topic_list:
            topic_list.append(j)

In [10]:
len(topic_list)

55

In [11]:
with open("./data/Challenge_Dataset/topics.txt",'w') as f:
    for i in topic_list:
        f.write(i+'\n')
f.close()

In [12]:
topic_tag_list = []
topic_tags = articles['vstopic'].dropna()
for tag in topic_tags:
    t = tag.strip('[').strip(']').strip().split(',')
    for i in range(len(t)):
        j = t[i].strip().replace("'",'')
        if j not in topic_tag_list:
            topic_tag_list.append(j)

In [13]:
with open("./data/Challenge_Dataset/topic_tags.txt",'w') as f:
    for i in topic_tag_list:
        f.write(i+'\n')
f.close()

In [14]:
vsproduct_list = []
products = articles['vsproduct'].dropna()
for product in products:
    t = product.strip('[').strip(']').strip().split(',')
    for i in range(len(t)):
        j = t[i].strip().replace("'",'')
        if j not in vsproduct_list:
            vsproduct_list.append(j)

In [15]:
product_list = []
products = articles['product'].dropna()
for product in products:
    t = product.strip('[').strip(']').strip().split(',')
    for i in range(len(t)):
        j = t[i].strip().replace("'",'')
        if j not in product_list:
            product_list.append(j)

In [16]:
with open("./data/Challenge_Dataset/products.txt",'w') as f:
    for i in product_list:
        f.write(i+'\n')
f.close()

In [17]:
with open("./data/Challenge_Dataset/product_tags.txt",'w') as f:
    for i in vsproduct_list:
        f.write(i+'\n')
f.close()

In [18]:
user_id = df['ip'].unique()

In [19]:
np.savetxt('./data/Challenge_Dataset/users.txt',user_id,fmt="%d")

In [20]:
d = {}
for i in user_id:
    d[i]=list(df[df['ip']==i]['tcm_id'])

In [21]:
recommended_together = []
for a_id in article_id:
    tmp = {}
    for k,v in d.items():
        if a_id in tmp:
            if a_id in v:
                tmp[a_id]=list(set(tmp[a_id]).intersection(set(v)))
        else:
            if a_id in v:
                tmp[a_id]=v
            else:
                tmp[a_id]=[]
    recommended_together.append(tmp)

In [22]:
recommended_together

[{'tcm:526-10787': ['tcm:526-12174', 'tcm:526-208086', 'tcm:526-10787']},
 {'tcm:526-10838': ['tcm:526-12174', 'tcm:526-208086', 'tcm:526-10838']},
 {'tcm:526-10849': ['tcm:526-10849', 'tcm:526-12174', 'tcm:526-208086']},
 {'tcm:526-116442': ['tcm:526-116442', 'tcm:526-12174', 'tcm:526-208086']},
 {'tcm:526-12167': ['tcm:526-12167', 'tcm:526-12174', 'tcm:526-208086']},
 {'tcm:526-12173': []},
 {'tcm:526-12174': ['tcm:526-12174', 'tcm:526-208086']},
 {'tcm:526-12180': ['tcm:526-12174', 'tcm:526-208086', 'tcm:526-12180']},
 {'tcm:526-13782': ['tcm:526-12174', 'tcm:526-208086', 'tcm:526-13782']},
 {'tcm:526-149043': ['tcm:526-12174', 'tcm:526-208086', 'tcm:526-149043']},
 {'tcm:526-153236': ['tcm:526-153236', 'tcm:526-12174', 'tcm:526-208086']},
 {'tcm:526-161967': []},
 {'tcm:526-164352': ['tcm:526-164352',
   'tcm:526-12174',
   'tcm:526-208086',
   'tcm:526-22867']},
 {'tcm:526-16645': ['tcm:526-16645', 'tcm:526-12174', 'tcm:526-208086']},
 {'tcm:526-18025': ['tcm:526-12174', 'tcm:526-

In [23]:
d_response_article = {}
for i in user_id:
    d_response_article[i]=list(df[(df['ip']==i) & (df['response']==1)]['tcm_id'])

In [24]:
also_response = {}
for a_id in article_id:
    l = []
    for k,v in d_response_article.items():
        if a_id in v:
            l+=v
    also_response[a_id]=list(set(l))

In [25]:
response_together = []
for a_id in article_id:
    tmp = {}
    for k,v in d_response_article.items():
        if a_id in tmp:
            if a_id in v:
                tmp[a_id]=list(set(tmp[a_id]).intersection(set(v)))
        else:
            if a_id in v:
                tmp[a_id] = v
            else:
                tmp[a_id] = []
    response_together.append(tmp)

In [26]:
word_count_dict = {}

In [27]:
def word_dict_generate(text):
    word_dict = {}
    words = []
    for line in text:
        line = line.strip().lower()
        #line = re.sub(r"[^A-Za-z0-9()<>:;,.!?\'\"]"," ",line)
        line = re.sub(r","," ",line)
        line = re.sub(r";"," ",line)
        line = re.sub(r"\'"," ",line)
        line = re.sub(r":"," ",line)
        line = re.sub(r"!"," ",line)
        line = re.sub(r"\."," ",line)
        line = re.sub(r"\/"," ",line)
        line = re.sub(r"\?"," ", line)
        line = re.sub(r"[\\d]"," ",line)
        line = re.sub(r"\'\'"," ",line)
        line = re.sub(r"\("," ",line)
        line = re.sub(r"\)"," ",line)
        line = re.sub(r"\<"," ",line)
        line = re.sub(r"\>"," ",line)
        line = nltk.word_tokenize(line)
        for w in line:
            if w not in stopwords.words("english") and len(w)>3:
                if w not in word_dict:
                    word_dict[w]=1
                else:
                    word_dict[w]+=1
                words.append(w)
        #words+=line
    return word_dict,words

In [28]:
word_count_dict,words = word_dict_generate(articles['text'])

In [29]:
len(word_count_dict),len(words)

(5359, 49263)

In [30]:
def computeTF(wordDict, words):
    tfDict = {}
    wordsCount = len(words)
    for word, count in wordDict.items():
        tfDict[word] = count / float(wordsCount)
    return tfDict

In [31]:
tf = computeTF(word_count_dict,words)

In [32]:
def computeIDF(text):
    N = len(text)
    idfDict = dict.fromkeys(text.keys(), 0)
    for word, val in text.items():
        if val > 0:
            idfDict[word] += 1
    for word, val in idfDict.items():
        if val!=0:
            idfDict[word] = math.log(N / float(val))
    return idfDict

In [33]:
idf = computeIDF(word_count_dict)

In [34]:
def computeTFIDF(tfDict, idfs):
    tfidf = {}
    for word, val in tfDict.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [35]:
tfidf = computeTFIDF(tf,idf)

In [36]:
word_list = []
for w,s in tfidf.items():
    if s>0.0005:
        word_list.append(w)

In [37]:
len(word_list)

2246

In [38]:
with open("./data/Challenge_Dataset/vocab.txt",'w') as f:
    for w in word_list:
        f.write(w+'\n')
f.close()

In [39]:
def index_set(s):
    i = 0
    s_map = {}
    for key in s:
        s_map[key] = str(i)
        i+=1
    return s_map

In [40]:
user_map = index_set(user_id)
article_map = index_set(article_id)
word_map = index_set(word_list)
topic_map = index_set(topic_list)
tag_map = index_set(topic_tag_list)
product_map = index_set(product_list)
topic_tag_map = index_set(topic_tag_list)
product_tag_map = index_set(vsproduct_list)

In [41]:
with open("./data/Challenge_Dataset/response_together.txt",'w') as f:
    for line in response_together:
        t = ''
        for k,v in line.items():
            for k in v:
                t=t+' '+article_map[k]
        f.write(article_map[k]+' '+t+'\n')
f.close()

In [42]:
with open("./data/Challenge_Dataset/recommended_together.txt",'w') as f:
    for line in recommended_together:
        t = ''
        for k,v in line.items():
            for k in v:
                t=t+' '+article_map[k]
        f.write(article_map[k]+' '+t+'\n')
f.close()

In [43]:
with open("./data/Challenge_Dataset/also_response.txt",'w') as f:
    for k,v in also_response.items():
        t = ''
        for k in v:
            t=t+' '+article_map[k]
        f.write(article_map[k]+' '+t+'\n')
f.close()

In [44]:
with open("./data/Challenge_Dataset/has_topic.txt",'w') as f:
    for index,row in articles.iterrows():
        g = ''
        if type(row['topic']) == str:
            topics = row['topic'].strip('[').strip(']').strip().split(',')
            for t in topics:
                s = t.strip().replace("'",'')
                g=g+' '+topic_map[s]
        f.write(g+'\n')
f.close()

In [45]:
#has_vstopic_tag
with open("./data/Challenge_Dataset/has_topic_tag.txt",'w') as f:
    for index,row in articles.iterrows():
        k = ''
        if type(row['vstopic']) == str:
            tags = row['vstopic'].strip('[').strip(']').strip().split(',')
            for t in tags:
                s = t.strip().replace("'",'')
                k=k+' '+topic_tag_map[s]
        #f.write(article_map[row['tcm_id']]+' '+k+'\n')
        f.write(k+'\n')
f.close()

In [46]:
#has_vsproduct_tag
with open("./data/Challenge_Dataset/has_product_tag.txt",'w') as f:
    for index,row in articles.iterrows():
        k = ''
        if type(row['vsproduct']) == str:
            tags = row['vsproduct'].strip('[').strip(']').strip().split(',')
            for t in tags:
                s = t.strip().replace("'",'')
                k=k+' '+product_tag_map[s]
        #f.write(article_map[row['tcm_id']]+' '+k+'\n')
        f.write(k+'\n')
f.close()

In [47]:
with open("./data/Challenge_Dataset/has_product.txt",'w') as f:
    for index,row in articles.iterrows():
        g = ''
        #print(article_map[row['tcm_id']]+' '+
        if type(row['product']) == str:
            products = row['product'].strip('[').strip(']').strip().split(',')
            for t in products:
                s = t.strip().replace("'",'')
                g=g+' '+product_map[s]
        f.write(g+'\n')
f.close()

In [48]:
with open("./data/Challenge_Dataset/dataset.txt",'w') as f:
    for index,row in df.iterrows():
        if len(articles[articles['tcm_id']==row['tcm_id']]['text'].values) > 0:
            text = articles[articles['tcm_id']==row['tcm_id']]['text'].values[0].strip().lower().split(' ')
            k = ''
            for w in text:
                if w in word_map:
                    k=k+' '+word_map[w]
            f.write(user_map[row['ip']]+'\t'+article_map[row['tcm_id']]+'\t'+k+'\n')
f.close()

### For each customer has 10 articles in test dataset

In [49]:
import random
with open('./data/Challenge_Dataset/dataset.txt','r') as f:
    i = 0
    t = f.readlines()
    j = len(t)
    l_train,l_test = [],[]
    
    d={}
    for line in t:
        c = line.split('\t')[0]
        if c in d:
            if len(d[c])<10:
                d[c].append(line)
                l_test.append(line)
            else:
                l_train.append(line)
        else:
            d[c]=[line]
            l_test.append(line)
f.close()

In [50]:
with open('./data/Challenge_Dataset/train.txt','w') as f:
    for l in l_train:
        f.write(l)
f.close()

In [51]:
with open('./data/Challenge_Dataset/test.txt','w') as f:
    for l in l_test:
        f.write(l)
f.close()